In [40]:
# Importando pacotes necessário

import pandas as pd
from datetime import datetime
import numpy as np
import re

In [2]:
# df_completo = pd.read_csv('TMDB_movie_dataset_v11.csv')
# df_completo.head(1)

In [71]:
# Puxando as colunas necessárias do dataset

TMDB = pd.read_csv('TMDB_movie_dataset_v11.csv',
                 usecols=['title','release_date','production_companies','genres','runtime','revenue'])
TMDB['release_date'] = pd.to_datetime(TMDB['release_date'], format='%Y-%m-%d').dt.year
TMDB['release_date'] = TMDB['release_date'].astype('Int64')
TMDB['key'] = TMDB['release_date'].astype(str)
TMDB['key'] = TMDB['key'].str[0:4] + TMDB['title'].str.lower()
TMDB = TMDB[['key','title','release_date','revenue','runtime','genres','production_companies']]

In [72]:
TMDB.columns

Index(['key', 'title', 'release_date', 'revenue', 'runtime', 'genres',
       'production_companies'],
      dtype='object')

In [75]:
awards = pd.read_csv('the_oscar_award.csv')
awards['key'] = awards['year_film'].astype(str)
awards['key'] = awards['key'] + awards['film'].str.lower()
awards = awards[['key','year_film','year_ceremony','ceremony','category','name','film','winner']]
awards.head()

,key,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927the noose,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927the last command,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927a ship comes in,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,19277th heaven,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927sadie thompson,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [76]:
df_comp = pd.merge(awards,df, on='key', how='left')
df_comp = df_comp[[
    'key',
    'year_film',
    'year_ceremony',
    'category',
    'name',
    'film',
    'winner',
    'title',
    'revenue',
    'runtime',
    'genres',
    'production_companies']]
df_comp.head()

,key,year_film,year_ceremony,category,name,film,winner,title,revenue,runtime,genres,production_companies
0,1927the noose,1927,1928,ACTOR,Richard Barthelmess,The Noose,False,NaN,NaN,NaN,NaN,NaN
1,1927the last command,1927,1928,ACTOR,Emil Jannings,The Last Command,True,NaN,NaN,NaN,NaN,NaN
2,1927a ship comes in,1927,1928,ACTRESS,Louise Dresser,A Ship Comes In,False,NaN,NaN,NaN,NaN,NaN
3,19277th heaven,1927,1928,ACTRESS,Janet Gaynor,7th Heaven,True,7th Heaven,0.0,110.0,"Drama, Romance",Fox Film Corporation
4,1927sadie thompson,1927,1928,ACTRESS,Gloria Swanson,Sadie Thompson,False,NaN,NaN,NaN,NaN,NaN


In [77]:
df_comp.isna().sum()

key                     4147
year_film                  0
year_ceremony              0
category                   0
name                      65
film                    4147
winner                     0
title                   5485
revenue                 1338
runtime                 1338
genres                  4098
production_companies    6061
dtype: int64

In [ ]:
# Próximo passo: dar explode no df_comp pra ter gêneros e production_companies separados por linhas
# production_companies:

df_comp['production_companies'] = df_comp['production_companies'].str.split(',')
df_comp_exp = df_comp.explode('production_companies')
df_comp_exp['production_companies'] = df_comp_exp['production_companies'].str.strip()

In [12]:
df_comp = df_comp_exp

In [13]:
df_comp['genres'] = df_comp['genres'].str.split(',')
df_comp_exp = df_comp.explode('genres')
df_comp_exp['genres'] = df_comp_exp['genres'].str.strip()

In [17]:
df_comp_exp.to_csv('df_completo.csv')

In [122]:
# Conferir colunas

print(df.columns)

Index(['title', 'release_date', 'revenue', 'runtime', 'genres',
       'production_companies', 'key'],
      dtype='object')


In [123]:
# Tirar linhas com coluna "release_date" vazia
df = df[df['release_date'].notnull()]

# Tirar linhas com coluna "title" vazia
df = df[df['title'].notnull()]

In [124]:
df.to_csv('TMDB_menor.csv')

In [125]:
# Criando tabela "companies", selecionando colunas e criando a coluna "chave"

cmp = df[['key','title','release_date','production_companies']]

In [126]:
cmp.head()

,key,title,release_date,production_companies
0,2010inception,Inception,2010,"Legendary Pictures, Syncopy, Warner Bros. Pict..."
1,2014interstellar,Interstellar,2014,"Legendary Pictures, Syncopy, Lynda Obst Produc..."
2,2008the dark knight,The Dark Knight,2008,"DC Comics, Legendary Pictures, Syncopy, Isobel..."
3,2009avatar,Avatar,2009,"Dune Entertainment, Lightstorm Entertainment, ..."
4,2012the avengers,The Avengers,2012,Marvel Studios


In [127]:
# Transformando valores separados por vírgula em várias linhas

cmp['production_companies'] = cmp['production_companies'].str.split(',')
companies = cmp.explode('production_companies')
companies['production_companies'] = companies['production_companies'].str.strip()

C:\Users\Pedro\AppData\Local\Temp\ipykernel_14416\1168703258.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmp['production_companies'] = cmp['production_companies'].str.split(',')


In [128]:
# Verificando se deu certo

companies.head()

,key,title,release_date,production_companies
0,2010inception,Inception,2010,Legendary Pictures
0,2010inception,Inception,2010,Syncopy
0,2010inception,Inception,2010,Warner Bros. Pictures
1,2014interstellar,Interstellar,2014,Legendary Pictures
1,2014interstellar,Interstellar,2014,Syncopy


In [129]:
# Tirar linhas com coluna "title" vazia
companies = companies[companies['production_companies'].notnull()]

In [130]:
# Exportando para arquivo

companies.to_csv('companies.csv')

In [134]:
# Criando a tabela gênero

genre = df[['key','title','release_date','genres']]
genre['genres'] = genre['genres'].str.split(',')
genre = genre.explode('genres')
genre['genres'] = genre['genres'].str.strip()

C:\Users\Pedro\AppData\Local\Temp\ipykernel_14416\846225277.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre['genres'] = genre['genres'].str.split(',')


In [135]:
genre.head()

,key,title,release_date,genres
0,2010inception,Inception,2010,Action
0,2010inception,Inception,2010,Science Fiction
0,2010inception,Inception,2010,Adventure
1,2014interstellar,Interstellar,2014,Adventure
1,2014interstellar,Interstellar,2014,Drama


In [136]:
genre.to_csv('genres.csv')